# 8. 감성 분석에 머신 러닝 적용하기

In [2]:
!pip install pyprind

In [1]:
%load_ext watermark
%watermark -u -d -v -p numpy,pandas,sklearn,nltk

last updated: 2019-08-17 

CPython 3.7.3
IPython 7.6.1

numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
nltk 3.4.4


In [1]:
import pyprind
import pandas as pd
import os
import numpy as np

### 영화 리뷰 데이터셋을 하나로 묶어서 CSV로 전환

In [ ]:
basepath = 'aclImdb'

labels = {'pos':1, 'neg':0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
for s in ('test', 'train'):
    for l in ('pos', 'neg'):
        path = os.path.join(basepath, s, l)
        for file in sorted(os.listdir(path)):
            with open(os.path.join(path,file),'r', encoding='utf-8') as infile:
                txt = infile.read()
            df = df.append([[txt, labels[l]]], ignore_index=True)
            pbar.update()
            
df.columns = ['review', 'sentiment']

데이터 프레임의 순서를 섞는다.

In [ ]:
import numpy as np

np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))

만들어진 데이터를 CSV 파일로 저장한다.

In [ ]:
df.to_csv('movie_data.csv', index=False, encoding='utf-8')

저장한 CSV 파일을 불러온다.

In [2]:
import pandas as pd

df = pd.read_csv('movie_data.csv', encoding='utf-8')
df.head(3)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
review       50000 non-null object
sentiment    50000 non-null int64
dtypes: int64(1), object(1)
memory usage: 781.3+ KB


## BoW(Bag of Word) 모델 소개

### 단어를 특성 벡터로 변환하기
CountVectorizer의 fit_trainsform 메서드를 호출하여 BoW 모델의 어휘 사전을 만들고, 다음 세 문장의 희소한 특성 벡터로 변환하자.
1. The sun is shining
2. The weather is sweet
3. The sun is shining, the weather is sweet, and one and one is two

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer()
docs = np.array(['The sun is shining',
                 'The weather is sweet',
                 'The sun is shining, the weather is sweet, and one and one is two'])
bag = count.fit_transform(docs)

In [4]:
print(count.vocabulary_)

{'the': 6, 'sun': 4, 'is': 1, 'shining': 3, 'weather': 8, 'sweet': 5, 'and': 0, 'one': 2, 'two': 7}


In [5]:
print(bag.toarray())

[[0 1 0 1 1 0 1 0 0]
 [0 1 0 0 0 1 1 0 1]
 [2 3 2 1 1 1 2 1 1]]


### tf-idf를 사용해 단어 적합성 평가하기
너무 자주 나오는(tf 높은)단어는 여기저기 많이 나오기 때문에 감성 분석에서 중요하지 않다.  
그러므로 적게 나오는 것을 높은 가중치를 주는 idf와 곱하여 tf-idf를 사용하게 된다.  
그렇게 되면 적당히 나오는 중요한 단어들에게 높은 가중치를 줄 수 있다.  

In [6]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                        norm='l2',
                        smooth_idf=True)
print(tfidf.fit_transform(count.fit_transform(docs)).toarray())

[[0.         0.43370786 0.         0.55847784 0.55847784 0.
  0.43370786 0.         0.        ]
 [0.         0.43370786 0.         0.         0.         0.55847784
  0.43370786 0.         0.55847784]
 [0.50238645 0.44507629 0.50238645 0.19103892 0.19103892 0.19103892
  0.29671753 0.25119322 0.19103892]]


In [7]:
tf_is = 3
n_docs = 3
idf_is = np.log((n_docs+1) / (3+1))
tfidf_is = tf_is * (idf_is + 1)
print('tf-idf of term "is" = %.2f' %tfidf_is)

tf-idf of term "is" = 3.00


In [8]:
tfidf = TfidfTransformer(use_idf=True, norm=None, smooth_idf=True)
raw_tfidf = tfidf.fit_transform(count.fit_transform(docs)).toarray()[-1]
raw_tfidf

array([3.38629436, 3.        , 3.38629436, 1.28768207, 1.28768207,
       1.28768207, 2.        , 1.69314718, 1.28768207])

tf-idf는 l2 정규화를 하여 사용한다.

In [9]:
l2_tfidf = raw_tfidf / np.sqrt(np.sum(raw_tfidf**2))
l2_tfidf

array([0.50238645, 0.44507629, 0.50238645, 0.19103892, 0.19103892,
       0.19103892, 0.29671753, 0.25119322, 0.19103892])

### 텍스트 데이터 정제
불필요한 텍스트를 제거

In [10]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

In [12]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(||D|P)', text)
    text = (re.sub('[\W]+', ' ', text.lower())+
            ' '.join(emoticons).replace('-',''))
    return text

In [13]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available:'

In [15]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

In [16]:
df['review'] = df['review'].apply(preprocessor) # apply는 func만

In [ ]:
#df['review'].map(preprocessor) # map은 func, dict, series 가능

### 문서를 토큰으로 나누기
-> 간단한 방법은 띄어쓰기를 기준으로 나누는 것이다.

In [19]:
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

단어를 변하지 않는 기본 형태인 어간으로 바꾸는 **어간 추출(stemming)** 방법을 사용한다.

nltk에 있는 **포터 어간 추출기(porter stemmer)** 알고리즘을 사용해보자.

In [20]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

In [21]:
tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

In [22]:
tokenizer_porter('runners like running and thus they run')

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']

모델에 훈련 시키기 전에 **불용어(stop-word)** 를 처리해줘야 한다.
> 불용어란 모든 종류의 텍스트에 아주 흔하게 등장하는 단어이다.

In [23]:
import nltk

In [24]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\재은\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [26]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')[-10:] 
if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

## 문서 분류를 위한 로지스틱 회귀 모델 훈련하기

In [27]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [28]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

In [ ]:
tfidf = TfidfVectorizer(strip_accents=None,
                       lowercase=False,
                       preprocessor=None)

param_grid=[{'vect__ngram_range':[(1, 1)],
            'vect__stop_words':[stop, None],
            'vect__tokenizer':[tokenizer, tokenizer_porter],
            'clf__penalty':['l1','l2'],
            'clf__C':[1.0, 10.0, 100.0]},
           {'vect__ngram_range':[(1, 1)],
           'vect__stop_words':[stop,None],
           'vect__tokenizer':[tokenizer, tokenizer_porter],
           'vect__use_idf':[False],
           'clf__penalty':['l1', 'l2'],
           'clf__C':[1.0, 10.0, 100.0]},
           ]

lr_tfidf = Pipeline([('vect', tfidf),
                    ('clf', LogisticRegression(solver='liblinear', random_state=0))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                          scoring='accuracy',
                          cv=5,
                          verbose=1,
                          n_jobs=1)